# Markov chains, Monte Carlo methods, pattern theory

$\quad$ These notes serve as background for an ongoing project with a good friend, [Connor Sempek](https://github.com/connorsempek). We are following [this](http://bit-player.org/2021/three-months-in-monte-carlo) blog post by [Brian Hayes](http://bit-player.org/about-the-author). Our current objective is to re-create Hayes' simulations of Ising model dynamics. The underlying canvas of the model is two-dimensional, so the simulation can be thought of as a way to generate a sequence of `100x100` pixel images, with pixel values either black or white. The Ising model dynamics update the current image by changing the value of one pixel at a time. The purpose of these notes is to get a better understanding of how long the simulation must be run so that, for a sparse enough subsequence of the images:

* each is sampled approximately from the [Gibbs probability measure](https://en.wikipedia.org/wiki/Gibbs_measure) governing the Ising model, and 

* these are nearly independent.

$\quad$ The latter property is important for a longer term goal of the project, which is to use these subsequences of images as a computer vision dataset. Importantly, the Gibbs measure we are sampling from always depends on a global temperature parameter $T$, leading to natural classification and regression tasks.

$\quad$ Here is a starting point for formulating a classification task. Suppose that we can simulate Ising model dynamics well at two _distinct_ temperatures. It feels natural to choose one of these, say $T_1$, above the critical temperature $T_c$ of the model, and the other, $T_2$ below it. We then create two image datasets $\mathcal{D}_1$ and $\mathcal{D}_2$ of equal size by simulating as described by the two points above. Merging these datasets to form what we call $\mathcal{D}$ leads to a natural classification problem: given an element $x \in \mathcal{D}$ chosen uniformly at random, determine whether it came from $\mathcal{D}_1$ or $\mathcal{D}_2$. 

$\quad$ The relevance of the Ising model to vision has been explored heavily, it seems. A lot of the text below draws from Govind Menon's [pattern theory](https://www.dam.brown.edu/people/menon/publications/pt-2020.pdf) notes, where a chapter is devoted to this "energetic" perspective applied to character recognition tasks. Menon's notes also reference a 1984 [paper](http://www.peterbeerli.com/classes/images/a/a1/Geman_geman_1984.pdf) which develops a way to perform image restoration and enhancement using the Ising model. This makes it a natural model to experiment with, and there are a few directions I'd like to explore longer term:

1. At the lower temperature, $T_2$, simulating the model is akin to an optimization problem: as $T \to 0$, the Gibbs measure concentrates on the set of lowest-energy states, or _ground states_. Any simulation designed to sample from a low-temperature Gibbs measure needs to navigate the energy landscape of the system from some starting point to a low altitude, with the temperature dictating a small range of desirable low altitudes $\equiv$ low energies. This is effectively an optimization problem with cost function corresponding to the energy landscape. There is also an optimization inherent to the process of training any neural network on the classification task suggested. The question I would like to explore, loosely, is whether there are useful connections between these optimization problems. 


2. We will train a simple convolutional architecture on the dataset $\mathcal{D}$. Whether it is through pooling layers, or the local averaging inherent in the convolution operation, these architectures seem to apply a kind of [coarse-graining](https://en.wikipedia.org/wiki/Coarse-grained_modeling) to input signals $x \in \mathcal{D}$. This has prompted investigation into the connections between deep learning and the [renormalization group](https://en.wikipedia.org/wiki/Renormalization_group) of physics. The latter feels like a kind of microscope for theoretical physicists $-$ the renormalization group formalizes a sequence of scaling transformations acting on a physical system, typically zooming out and thereby coarse-graining the preceding image of the system. This parallels feeding the image of the system into a "deep" sequence of convolutional layers. Two references for this are a 2013 [paper](https://arxiv.org/abs/1301.3124) of Cédric Bény, and a 2014 [paper](https://arxiv.org/abs/1410.3831v1) of Pankaj Mehta and David Schwab, and I imagine there is more recent work on this to explore. Dataset $\mathcal{D}$ is both an image dataset and a natural input to the renormalization group. It will give us an opportunity to better understand these connections, in particular we can start by repeating the experiments the latter paper runs on the Ising model.

3. A goal less directly related to ML is to augment the simulation to reproduce the simulations of Lubetzky and Sly on _information percolation_ ([main article](https://math.nyu.edu/~eyal/papers/cutoffbhZd.pdf) with visualization, and [this](https://cims.nyu.edu/~eyal/papers/cutoffbh_expo.pdf) is a shorter expository version, still technical). Information percolation stacks each time-slice of the simulation described above (at some fixed temperature) into a 'space-time cylinder'. Multi-colored edges or 'bonds' are drawn _between_ pixels in successive time-slices, based on how the simulation dynamics progress. The color of the bonds reflects different possibilities for how the update step is executed at a given pixel: sometimes, the randomness involved in the update is completely independent of everything else, and this is how the simulation gradually loses its memory of its starting point. The information percolation process outputs a graph or network inside the space-time cylinder encoding all aspects of the simulation. The relevance of information percolation is that becoming decoupled from the simulation starting point has the interpretation that the top slice of the space-time cylinder is _not_ connected to the bottom slice through the information perecolation process. This is a way to graphically represent the 'mixing' undergone by the Ising dynamics. Lubetzky and Sly use this perspective to deduce lower bounds on how long the simulation dynamics must be run in order to sample effectively from the Gibbs measure of the Ising model. So this is a tool relevant to both bullet points above, and could look cool to simulate and visualize.

___

_references_

* Krauth (2006) _statistical mechanics: algorithms and computations_

* Menon (2020) _[pattern theory](http://www.dam.brown.edu/people/menon/publications/pt-2020.pdf)_

___

$\quad$ The Monte Carlo method is increasingly becoming part of the discipline it is meant to study. The Monte Carlo method is a statistical, _"almost experimental"_, approach to computing integrals using random positions, called __samples__. Below, we introduce some sampling techniques for continuous and discrete variables. We also discuss the basic principles of statistical data analysis: how to extract results from well-behaved simulations. We also discuss at length the simulations where something goes wrong. 

$\quad$ In probabilistic terms, the integrals we desire to approximate correespond to the expected value of some observable, $\mathcal{O} : \mathcal{S} \to \mathbb{R}$. Here we assume some implicit probability measure $\mu$ on $\mathcal{S}$, the latter called the __state space__. Formally, the expected value $\mathbb{E} \mathcal{O}$ corresponds to the integral
$$
\mathbb{E} \mathcal{O} = \int_{ \mathcal{S} } \mathcal{O}(s) \,  \mu( \textrm{d} s )
$$
We write this expectation either as $\mathbb{E} \mathcal{O}$, or as $\langle \mathcal{O} \rangle$, the latter notation more common in physics.  

$\quad$ We want to simulate most of the objects discussed throughout the notes, so we lose no generality in assuming that $\mathcal{S}$ is finite. $\mathcal{S}$ will be treated as a continuous object when it is more natural, or becomes convenient.

> I am tempted to jump to conclusions in the following way: first, note that if $\mathcal{S}$ is finite with $\# \mathcal{S} = M$, then $\mu$ corresponds to its probability mass function $p = (p_1, \dots, p_M)$. In this case, the above integral becomes the $\sum_{i=1}^M o_i p_i$ for a given vector $o = (o_1, \dots, o_M)$. The conclusion I jumped to was that approximating this sum happens first by estimating each $p_i$ through a statistic (like a histogram) $\hat{p}$ of some long-running simulation, and then taking a dot product of $\hat{p}$ with $o$. 

> To be clear, in both direct sampling and Markov chain sampling, it seems instead that our approximation comes from taking measurements of a system as it evolves, and only after it has been allowed to evolve for some time. The measurements applied to each state of $\mathcal{S}$ are encoded in the measurement of the system, and the probability vector $p$ (of $\mu$) is encoded in the system evolution itself. 

$\quad$ We wish to distinguish between two distinct sampling approaches: __direct sampling__ and __Markov chain sampling__. We postpone the definition of a Markov chain until later; for now it is understood as a random walk.

## ... direct sampling ...

_(Krauth $\S$ 1.1.1)_

$\quad$ Krauth explains direct sampling through a game played with small pebbles. The playing field is a large circle is inscribed into a large square, both drawn in the sand. In direct sampling, one has 'access' to the measure $\mu$ sampling from $\mathcal{S}$. In the context of the game, we will assume we can directly sample from $\mu$, which denotes the uniform measure on the square 
$$
\mathcal{S} \equiv \textsf{square} = [-1,1]^2   \,,
$$ 
a 'birds-eye' view of the playing field. Not only this, but we assume that we can also sample directly from $\mu$ multiple times, _independently_. 

> The observable considered in the game is related to its goal, namely to compute an approximation of the number $\pi$. The observable $\mathcal{O} : \mathcal{S} \to \mathbb{R}$ is the indicator function of the unit disc, 
$\textsf{disc} = \{ s = (x,y) \in \textsf{square} : x^2 + y^2 \leq 1 \}$
$$
\mathcal{O} \equiv \mathbf{1}_{ \textsf{disc}} =
\begin{cases}
1 & \quad s \in \textsf{disc} \\
0 & \quad \textrm{otherwise}
\end{cases} 
$$

$\quad$ A player throws pebbles at the square, successively. Through the direct sampling, these land uniformly at random. The sampling is performed independently across draws from $\mu$. In particular, no pebble throw ever misses the square. Let $S_1, \dots, S_N$ denote the collection of locations sampled from $\textsf{square}$ in the above procedure: each $S_i$ is a random ordered pair $S_i = (x_i, y_i) \in \textsf{square}$, where all coordinates are mutually independent, $\textrm{Unif}([-1,1])$ random variables.




$\quad$ To describe how the samples $S_i$ are used to approximate the value of $\pi$, we introduce the __empirical measure__ $\mu_N^{\textrm{direct}}$, a random probability measure on $\mathcal{S}$, defined to encode the locations $S_1, \dots, S_N$: 
$$
\mu_N^{\textrm{direct}} := \frac{1}{N} \sum_{i=1}^N \delta_{S_i},
$$
where in general $\delta_S$ denotes the delta mass at the random point $S$.  Measures are in a sense dual to sets, and the __delta mass__ $\delta_X$ is a measure acting on sets in the following way: given measurable $B \subset \textsf{square}$, we have
$$
\delta_S = 
\begin{cases}
1 & \quad \textrm{ if } S \in B, \\
0 & \quad \textrm{ otherwise. }
\end{cases}
$$
The empirical measure $\mu_N^{\textrm{direct}}$, by definition, acts on sets $B \subset \mathcal{S}$ by reporting the fraction of the direct samples $S_i$ landing in $B$. 

$\quad$ There is a sense in which the measures $\mu_N^{\textrm{direct}}$ 'converge' to $\mu$, and the game, or simulation, is designed to exploit this. Let us enumerate the states of $\mathcal{S} = \{s_1, \dots, s_M \}$. In practice, the size of $M$ will be determined by the 'resolution' of the floating points used in sampling. In accordance with the observable $\mathcal{O}$, the set we choose for $B$ is $\textsf{disc}$. This is because
$$
\mu_N^{\textrm{direct}}( \textsf{disc} ) = \frac{1}{N} \sum_{i=1}^N \delta_{S_i}( \textsf{disc} ) \equiv \frac{1}{N} \sum_{i=1}^M \mathcal{O}(S_i)
$$
The sense in which $\mu_N^{\textrm{direct}}$ converges to $\mu$ implies, in particular, that the sequence of numbers $\mu_N^{\textrm{direct}}(\textsf{disc})$ converges to $\mu( \textsf{disc} )$ as $N \to \infty$. This convergence can also be seen as a consequence of a law of large numbers. 


In the present context, the relevant subset of $\textsf{square}$ is the disc of radius one, centered at the origin. We refer to this as $\textsf{disc}$. The output of the first algorithm below is 

$$
4 \cdot \mu_N^{\textrm{direct}}(\textsf{disc}) \approx 4 \cdot \mu( \textsf{disc}) \equiv 4 \cdot \frac{ \pi}{4} 
$$

Thus we are approximating $\pi$ by approximating $\pi/4$, which is the probability that a pebble sampled from $\mu$ lands in $\textsf{disc}$.

In [ ]:
import numpy as np
import sys
from numpy.random import default_rng

rng = default_rng()

The next function embodies the 'direct' sampling we supposedly have access to, with the independence assumptions described above. 

___

### method ... Unif

In [ ]:
# uniform r.v. over interval [0,1]
def Unif():
    return rng.uniform()

___

Algorithm 1.1 is translated from pseudocode (as it is presented in Krauth) to Python.

___

### Algorithm 1.1 ... direct_pi

In [ ]:
def direct_pi(N = 4000):

    N_hits = 0

    """
    the for loop is basically about computing the sum in
    \mu_N(disc), stored in N_hits
    """

    for i in range(N):

        # initialize Unif([-1,1]) r.v.'s
        x = 2 * Unif() - 1
        y = 2 * Unif() - 1

        rad_sq = (x ** 2) + (y ** 2)

        if rad_sq <= 1:

            N_hits += 1
    """
    \mu_N(disc) is then computed by dividing by N
    """
    ratio = N_hits / N

    return 4 * ratio

___

In [ ]:
direct_pi()

3.134

In Krauth, Table 1.1 gives results of five runs of the above with `N = 4000`. We return to this table later to compare with Monte Carlo methods. 

| run | `N_hits` | estimate of $\pi$ |
| -------- | -------- | --- |
| 1 | 3156 | 3.156 |
| 2 | 3150 | 3.150 |
| 3 | 3127 | 3.127 |
| 4 | 3171 | 3.171 |
| 5 | 3148 | 3.148 |

Krauth remarks that none of the results of this table have fallen in the error bounds known since Archimedes. Another remark of Krauth: _"The people adopt a sensible rule: they decide on the total number of throws, before they start the game."_ (This is as in the above python code.) _"They understood that one must not stop a stochastic calculation simply because the current result appears accurate, nor should they continue to play because the answer they get isn't close enough to their idealized target."_



## ... Markov-chain sampling ...

_( Krauth $\S$ 1.1.2 )_

$\quad$ We now consider a new game being played on the same field, with different rules. We again take a birds-eye view of the playing field, so our state space is again $\mathcal{S} = \textsf{square}$. The game starts with a person at the corner of the helipad, corresponding to coordinates $(1,1)$ in $\mathcal{S}$. In Krauth's description, in each coordinate, the walker takes a step via adding independent, $\textrm{Unif}([-\delta, \delta])$ random variables. Thus it is possible for a next step which takes the person outside the square. If this is the case, instead of taking the step, the walker does nothing. This step still 'counts' in the aggregation we will perform. Krauth describes this procedure in terms of the pebbles: the person takes another pebble from their bag and places it on top of the one pebble (or perhaps pebble pile) marking their current location.

$\quad$ After an 'in-bounds' step is taken, the walker places a pebble at their current position. Each walker is equipped with a bag of infinitely many pebbles. These two cases describe the way the new game is played. We make one modification to this, which is that 

$\quad$ In the present setting, the __samples__ $X_1, \dots, X_N$ are not independent, but rather have the structure of a Markov chain, specifically a random walk with independent increments. Let us use $\xi_1, \dots, \xi_N$ denote these independent increments. In Krauth's description, each of these is a $\textrm{Unif}([-\delta, \delta])$ random variable. For reasons that Krauth discusses later, and we will comment on this, we instead assume that each $\xi_j$ is a discrete random variable uniformly distributed on the two-element set $\{ -\delta, \delta \}$. We have distinguished the starting point of the sampling at $X_0 = (1,1)$. Following the above description, one then has
$$
X_1 = 
\begin{cases}
X_0 + \xi_1\,, & \quad \textrm{ if } X_0 + \xi_1 \in \textsf{square}\\
X_0 & \quad \textrm{ otherwise }
\end{cases}
$$

and in general,

$$
X_j = 
\begin{cases}
X_{j-1} + \xi_j\,, & \quad \textrm{ if } X_{j-1} + \xi_j \in \textsf{square}\\
X_{j-1} & \quad \textrm{ otherwise }
\end{cases}
$$

for all $j = 1, \dots, N$.

$\quad$ For this new sequence of random $X_i$, we can define the analogous object to $\mu_N^{\textrm{direct}}$, 
$$
\mu_N^{\textrm{Markov}} := \frac{1}{N} \sum_{i=1}^N \delta_{X_i}
$$

The algorithm below returns an approximation to $\pi$ via the same kind of ratio

$$
\pi \approx 4 \cdot \mu_N^{\textrm{Markov}}(\textsf{disc}) 
$$

$\quad$ One might be tempted to use random initial conditions, assigning independent $\textrm{Unif}([-1,1])$ random variables to the two coordinates. We avoid this because the Markov chain methods become most useful when no direct sampling method exists. It is for exactly this reason that we also avoid directly sampling from $\textrm{Unif}([-\delta, \delta])$ random variables: such objects are a linear transformation away from directly sampling from the $\textrm{Unif}([-1,1])$ distribution, perhaps at lower resolution. 

$\quad$  Consequently, our Markov-chain sampling simulations start at the corner $(1,1)$ for concreteness. They can also be assumed to start from where a previous simulation left off. Following this convention throughout these notes, we usually focus on going from configuration $i$ to configuration $i+1$. This is the defining property of (discrete-time) Markov chains: the transition probabilities of the process depend only on the current position. 



___

### Algorithm 1.2 ... markov-pi (unmodified)

In [ ]:
def markov_pi( delta = .05, N = 40000 ):

    # initial conditions
    x = 1

    y = 1

    N_hits = 0

    num_rej_moves = 0 # number of "rejected" moves

    for i in range(N):

        # get coordinate increments
        del_x_big = 2 * Unif() - 1

        del_y_big = 2 * Unif() - 1

        del_x = delta * del_x_big

        del_y = delta * del_y_big

        #print(del_x, del_y)

        # potential new coordinates
        x_new = x + del_x

        y_new = y + del_y

        # to determine if step leads into disc
        rad_sq_new = ( x_new ** 2 ) + ( y_new ** 2 )

        # ditto for square
        x_in = x_new <= 1 and x_new >= -1

        y_in = y_new <= 1 and y_new >= -1

        in_square = x_in and y_in 

        # if move is accepted
        if in_square:

            x = x_new

            y = y_new

        else:

            num_rej_moves += 1

        # if in disc, +1 to number of hits
        if rad_sq_new <= 1:

            N_hits += 1

    # we add in this output to 
    # better understand how to pick 
    # delta
    print( "fraction of successful moves: ", 1 - ( num_rej_moves / N) )

    return 4 * ( N_hits / N )


___

In [ ]:
markov_pi()

fraction of successful moves:  0.9751


3.0945

$\quad$ Let us discuss how to choose the  'throwing range' is $\delta$. It is akin to (largest possible) step size for the walker. Roughly, if $\delta$ is too small, the acceptance rate may be high, but the claim is that the walk will not be able to explore enough to well-approximate the integral. Choosing $\delta$ too large means the walk will have a hard time making successful moves, also hindering exploration. Krauth: _"The time-honored rule of thumb consists in choosing $\delta$ neither too large, nor too small, so that the acceptance rate turns our to be on the order of $\frac{1}{2}$."_ I didn't observe results agreeing with their rule of thumb. I found much more success with $\delta = 0.05$ versus their recommendation of $0.3$. I did however observe the approximation worsen as the step size was taken larger past $\delta = 1$. 

___

### Algorithm 1.2' ... markov-pi (modified)

In [ ]:
# in this modification, steps are no longer uniform  

def markov_pi_modified( delta = .05, N = 40000 ):

    # initial conditions
    x = 1

    y = 1

    N_hits = 0

    num_rej_moves = 0 # number of "rejected" moves

    for i in range(N):
        """
        we use uniform random variables, out of convenience but not to
        determine coordinate steps directly with these. Instead, we 
        extract a pair of fair coin tosses from two uniform random
        variables. We use these coin tosses only to determine the sign
        of the increment in each coordinate. 
        """

        # generate indep uniforms
        U_1 = Unif()

        U_2 = Unif()

        # use these as coin tosses to determine 
        # del_x...

        if U_1 <= 0.5:
        
            del_x = delta

        else:

            del_x = (-1) * delta

        if U_2 <= 0.5:

            del_y = delta

        else:

            del_y = (-1) * delta

        # potential new coordinates
        x_new = x + del_x

        y_new = y + del_y

        # to determine if step leads into disc
        rad_sq_new = ( x_new ** 2 ) + ( y_new ** 2 )

        # ditto for square
        x_in = x_new <= 1 and x_new >= -1

        y_in = y_new <= 1 and y_new >= -1

        in_square = x_in and y_in 

        # if move is accepted
        if in_square:

            x = x_new

            y = y_new

        else:

            num_rej_moves += 1

        # if in disc, +1 to number of hits
        if rad_sq_new <= 1:

            N_hits += 1

    # we add in this output to 
    # better understand how to pick 
    # delta
    print( "fraction of successful moves: ", 1 - ( num_rej_moves / N) )

    return 4 * ( N_hits / N )


___

In [ ]:
markov_pi_modified()

fraction of successful moves:  0.953025


3.1668

___

__Q__ $\quad$ In both the Glauber and Metropolis dynamics for the evolution of the Ising model, the spin variable to be updated ( i.e. the vertex $v \in \mathbb{T}$ sampled ) is chosen uniformly, through _direct sampling_. What if this is instead done through Markov-chain sampling? 

Also: instead of rejecting the moves outside the box, we can play with periodic boundary conditions for the walks in both simulations.
___

## ... discussion ...  

$\quad$ The Monte Carlo 'games' described above epitomize the two basic approaches to sampling from a probability measure $\mu$ on some space $\mathcal{S}$. Both approaches to the sampling evaluate an observable $\mathcal{O} : \mathcal{S} \to \mathbb{R}$. In our examples, $\mathcal{S} = \textsf{square}$, and the observable $\mathcal{O}$ is the indicator function of the disc within the square:
$$
\mathbf{1}_{\textsf{disc}}(s) = \begin{cases}
1 & \quad s \in \textsf{disc} \\
0 & \quad \textrm{otherwise}
\end{cases} \,, \quad \mathbf{1}_{\textsf{disc}} : \textsf{square} \to \mathbb{R} 
$$

In either sampling case, we can define $\mathcal{O}_i := \mathcal{O}(X_i)$, and note that one evaluates

$$
\frac{N_{\textrm{hits}}}{N} = \frac{1}{N} \sum_{i=1}^N \mathcal{O}_i \approx \langle \mathcal{O} \rangle,
$$

where the brackets $\langle \, \cdot \, \rangle$ denote the expected value of $\mathcal{O}$ taken with respect to the probability measure $\mu$, which in this case is uniform on $[-1,1]^2$. 

$\quad$ Krauth remarks that $\langle \mathcal{O} \rangle$ is an integral,
$$
\langle \mathcal{O} \rangle = \int_{-1}^1 \textrm{d} x \int_{-1}^1 \textrm{d}y \, \mu (x,y) \mathcal{O}(x,y)
$$
In his notes, Krauth includes a denominator on the right-hand side above, it is:
$$
\int_{-1}^1 \textrm{d} x \int_{-1}^1 \textrm{d}y \, \mu (x,y) 
$$

My understanding of $\mu(x,y)$ is that it is used to denote the density of probability measure $\mu$, so it integrates to one. Within the above bounds of integration, it can be taken as a constant, namely $1/4$, which is the reciprocal of the area of $[-1,1]^2$. Regardless, there is an integral to be computed, and this is an aspect Krauth remarks on next. 

$\quad$ The density $\mu(x,y)$ doesn't appear in the approximation $\tfrac{1}{N} \sum_{i=1}^N \mathcal{O}_i$, Krauth: _"...rather than being evaluated, it is sampled. This is what defines the Monte Carlo method."_ Krauth goes on to point out that the multiple integrals also disappear. _"This means that the Monte Carlo method allows the evaluation of high-dimensional integrals, such as appear in statistical physics and other domains, if we can only think of how to generate the samples."_

$\quad$ The multiple integral above is integration in two variables, so it doesn't feel like it is genuinely a 'high-dimensional' integral to compute. On the other hand, we are already know of a kind of Markov chain on a high-dimensional space, namely the Glauber dynamics for the Ising model on $\mathbb{T}$, the two-dimensional torus $( \mathbb{Z}\, /\, (n\cdot \mathbb{Z}) )^2$.  All in all, it seems that one is approximating an integral in an arbitrary number of dimensions (number of spins in the case of Ising model) through what is essentially a path integral. The path integral is a sum of observables of the system, as the system evolves. 

$\quad$ Krauth: _"Direct sampling, the approach inspired by the game, is like pure gold: a subroutine provides an independent hit at the distribution function $\pi$. Notwithstanding thee randomness in the problem, direct sampling, in computation, plays a role similar to exact solutions in analyitcal work, and the two are closely related. In direct sampling, there is no throwing-range issue, no worrying about initial conditions, and there is a straightforward error analysis -- at least if $\mu$ and $\mathcal{O}$ are well behaved. Many successsful Monte Carlo algorithms contain exact sampling as a key ingredient."_

$\quad$ _"Markov-chain sampling, on the other hand, forces us to be much more careful with all aspects of our calculation. The critical issue here is the correlation time, during which the position of the walker retains a memory of the starting configuration. This time can become astronomical. In the usual applications, onee is often satisfied with a handful of independent samples, obtained through week-long calculations, but it can require much throught and experience to ensure that even this modest goal is achieved. We shall continue our discussion of these methods in a later section. We first look briefly at the history of stochastic computing."_

before this, we draw background on Markov chains and Gibbs measures from Menon. 


## ... Markov chains ...

_(Menon $\S$ 1.1, 1.2)_

$\quad$ We model the world as a space of random signals. The world is filled with observers, which record part of these signals and then form inferences about the world. The process of inference is modeled with a Bayesian paradigm. We assume an observer has a stochastic model capable of generating signals similar to the true signal. Inference reduces to tuning the model parameters to obtain the best match between the generated and observed signals.

$\quad$ These notes explore stochastic models of progressively increasing complexity, in tandem with numeraical algorithms for optimization. The simplest class of stochastic models we consider are Markov chains. Suppose we are given a finite set $\mathcal{S}$ called the **state space** of the Markov chain. This will sometimes (such as in text-modeling instances) be called the **alphabet**, in which case we may use $\mathcal{A}$ in place of $\mathcal{S}$. 


___

### **Definition.** ( stochastic process )

$\quad$ A **stochastic process** is a collection of random variables.
___

$\quad$ We study a stochastic process $(X_t)_{t \in \mathbb{N}}$ taking values in the given finite set $\mathcal{S}$. Thus, the temporal (given by the index $t$ in the stochastic process) and the spatial (given by the state value in $\mathcal{S}$ of the process) aspects of this process are discrete. 

___

### **Definition.** ( Markov chain )

$\quad$ The stochastic process $(X_t)_{t \in \mathbb{N}}$ is a **Markov chain** if for all $t$, $\mathbb{P} ( X_{t +1} | X_1, \dots, X_t ) = \mathbb{P} (X_{t+1} | X_t )
$.

___

$\quad$ As the above definition indicates, the modeling assumption in Markov process theory is that the future of the process depends on the current state, but not the past. This is the probabilistic analogue of the idea of Newtonian determinism, and is encoded in the _forward equation_ below. Stationary Markov chains are thus characterized by their __transition matrix__ 
$$
Q(s, \tilde{s}) = \mathbb{P}( X_2 = \tilde{s} | X_1 = s )\,,
$$ 
with entries indexed by pairs of states $s, \tilde{s} \in \mathcal{S}$. Another interpretation of the Markov property is that the future and past are independent, conditional on the present. We typically simplify the structure of Markov chains with the following assumption.



___

### **Definition.** ( stationary Markov chain )

$\quad$ The Markov chain $(X_t)$ is **stationary** if $\mathbb{P}(X_{t+1} | X_t ) = \mathbb{P} (X_2 | X_1 )$ for all $t \in \mathbb{N}$. 
___

$\quad$ The most basic statistic associated to the Markov chain is the pmf( $\equiv$ probability mass function ), or law of the random variable $X_t$. The mass assigned to state $s \in \mathcal{S}$ under this law is denoted
$$
m_t(s) := \mathbb{P}(X_t = s) \,,
$$
and one obtains the following recurrence relation using the so-called law of total probability, along with the definitions of $m_t$ and $Q$:
$$
\begin{align}
m_{t+1}(\tilde{s}) &= \mathbb{P} ( X_{t+1} = \tilde{s}) \\
&= \sum_{s \in \mathcal{S}} \mathbb{P}( X_{t+1} = \tilde{s}, \, X_t = s ) \\
&= \sum_{s \in \mathcal{S}} \mathbb{P} (X_{t+1} = \tilde{s} | X_t = s ) \mathbb{P}(X_t = s)  \\
&= \sum_{s \in \mathcal{S}} Q(s,\tilde{s}) m_t(s),
\end{align}
$$
when we assume the chain is stationary. This calculation yields the **forward equation** 
$$
m_{t+1} = m_t Q , \quad t \in \mathbb{N}
$$
using the convention that the pmf $m_t$ of $X_t$ is regarded as a row vector.  The forward equation is linear and explicitly solvable when the chain is stationary. We proceed inductively to find
$$
m_t = m_1 Q^t , \quad t \in \mathbb{N} \,,
$$
and one of the central questions in Markov chain theory is to understand the behavior of $m_t$ as $t\to\infty$. This question explains the importance of the eigenvalue spectrum of $Q$ to Markov chain theory: diagonalizing $Q$ is an efficient way to compute $Q^t$ for $t$ large, and the spectrum of $Q$ largely governs the long term behavior of the system. 


## ... equilibrium measures, ergodicity, random walks ...

_(Menon $\S$ 1.2)_

___

### **Definition.** ( equilibrium / stationary distribution )

$\quad$ Let $Q$ be the transition matrix of a stationary Markov chain $X = (X_t)$ with state space $\mathcal{S}$, and let $m$ be a probability measure on $\mathcal{S}$, whose pmf we also denote $m$. The measure $m$ is an **equilibrium distribution** or equivalently, a **stationary distribution** for $X$ if 
$$
m = m Q,
$$
and if $m$ satisfies the normalization conditions $m(s) \geq 0$ for all $s \in \mathcal{S}$ and $\sum_{s \in \mathcal{S}} m(s) =1$.

___

$\quad$ The above definition is of central importance to our simulation goals. The reason is that, for Markov chains with a unique stationary distribution $m$, the measures $m_t \equiv \mathbb{P}(X_t = \cdot )$ converge to $m$, in an appropriate sense, as $t \to \infty$. When we simulate a Markov chain, we are sampling from $m_t$, and we want to run the simulation long enough (taking $t$ large enough) so that $m_t \approx m$. This is how a Markov chain Monte Carlo (MCMC) simulation can be used to approximate sampling from $m$. In practice, instead of starting with some Markov chain dynamics, we will always start with a measure $m$ on $\mathcal{S}$, which we would like to sample from.

$\quad$ The MCMC method then requires _finding_ a stationary, ergodic Markov chain $X$ with state space $\mathcal{S}$, whose stationary measure coincides with $m$. This is how $X$ is used to simulate sampling from $m$. 

$\quad$ In Krauth's pebble games, there was some discussion of initial conditions of a Markov chain: in the second pebble game, the person playing always had to start at a prescribed corner of the square. These are _deterministic_ initial conditions. In practice, we agreed to assume simulations will start deterministically, or from where a previous simulation left off. In theory, it is useful to have a mathematical object associated to the __initial conditions__ of a Markov chain: a probability measure on $\mathcal{S}$, denoted $m_0$. When initial conditions are deterministic, and start at some distinguished state $s_* \in \mathcal{S}$, the probability measure $m_0$ corresponds to the $\delta$-mass $\delta_{s_*}$. When $m_0$ is general, it models starting from where some simulation left off, whose information we have no (or only partial) access to. 

$\quad$ The first equation in the above definition can be understood as describing the evolution of a Markov chain $X$ whose initial conditions are the stationary distribution $m$ associated to $X$. In this case $m_t = m$ for all $t$ if $m_1 = m$. That is, while the values of $X_t$ change with time (it's a stochastic process) its probability mass function does not. Thus the equilibrium distribution of a Markov chain captures the intuitive idea of a dynamic equilibrium . 

$\quad$ Considering the task of computing $m$ given $Q$: observe that $m$ is a left eigenvector of $Q$ with eigenvalue $1$. Further, $1$ must always be an eigenvalue of $Q$ since it corresponds to the right eigenvector $(1, \dots, 1)^T$. Thus, if we know how to solve for eigenvectors, we can determine $m$. Menon: _"What is interesting in practice is the converse: for large transition matrices, say when $\# \mathcal{S}$ is $10^6 \times 10^6$, it is more efficient to use Markov chains to approximate $m$, than to use naive linear algebra This observation plays an important role in web crawlers and search engines."_

$\quad$ Returning to the main question of the Ising simulation: knowing how long to run the MCMC simulation for, or knowing how quickly $m_t$ converges to $m$ is related to how fast the Markov chain _mixes_. We now explore two examples of Markov chains whose state space is the symmetric group $\mathfrak{S}_{52}$. Each group element can be thought of as a possible shuffling of a standard deck of playing cards. The group identity of $\mathfrak{S}_{52}$ corresponds to some distinguished initial ordering. For each of the two shuffling methods described, the main question can be rephrased as: _"how many shuffles does it take to obtain a truly random (uniform) distribution if one begins with a perfectly ordered deck of cards?"_  The following term will be used. 

___

### **Definition.** ( ergodic Markov chain )

$\quad$ A stationary Markov chain $X$ with state space $\mathcal{S}$ and transition matrix $Q$ is **ergodic** if it has a unique equilibrium distribution. A sufficient condition for ergodicity of $X$ is if for every pair $s, \tilde{s} \in \mathcal{S}$ there is a positive integer $j(s,\tilde{s})$ such that $Q^{j(s,\tilde{s})} >0 $.
___

$\quad$ Many other conditions are known that ensure ergodicity. For example, if all terms of $Q$ are strictly positive, it is possible for any state to get to any other state, and this ensures that the Markov chain is ergodic. This assumption is too restrictive in practice, since we are mainly intersted in Markov chains making 'local' moves such as random walks in a large state space. This behavior is captured by the sufficient condition given in the above definition. Intuitively, it means that every state can visit every other state given enough time. We assume this property always holds.

$\quad$ Physicsts define the term ergodic ( describing Markov chain $(X_t)_{t\in \mathbb{N}}$ ) to mean the following.

$$
\lim_{T \to \infty} \sum_{t =1}^T \mathcal{O}(X_t) = \mathbb{E}_m\mathcal{O},
$$

where $m = m Q$ and $\mathcal{O}: \mathcal{S} \to \mathbb{R}$ is arbitrary. 

$\quad$ This formulation has the interpretation that a Markov chain is ergodic when the left-hand side, the _time-average_, is equal to the _spatial-average_ (with respect to the equilibrium measure), which is the right hand side. For Markov chains ergodic in the first sense, the second notion of ergodicity follows from an _ergodic theorem_. Menon notes the following bottlenecks and simplifications:

* $\mathcal{S}$ may be very large. For example (in shuffling dynamics we soon describe), $\mathcal{S}$ may be the permutation group. 

* $Q$ may be a sparse matrix.

* As discussed previously, we often don't need $m$ explicitly, we instead want to evaluate $\mathbb{E}_m \mathcal{O}$, where $\mathcal{O}: \mathcal{S} \to \mathbb{R}$ is some observable. 

___

### **Example.** ( simple random walk ) 

$\quad$ Let $G = (\textrm{V},\textrm{E})$ be a finite graph with vertex set $\textrm{V}$ and edge set $\textrm{E}$. Given $x,y \in \textrm{V}$, write $x \sim y$ if the edge $\{ x, y \}$ lies in $\textrm{E}$. Let $\textrm{deg}(x) \triangleq \sum_{x \sim y } 1$ be the degree of vertex $x$. A **simple random walk** on $G$ is a Markov chain with state space $\textrm{V}$ and transition matrix
$$
Q(x,y)
    =
        \begin{cases}
        \frac{1}{\textrm{deg}(x)}, & \quad \text{ if } x \sim y \\
        0 & \quad \text{ otherwise. }
        \end{cases}
$$
We leave the initial conditions ambiguous in this definition. 

___

___
### **Example.** ( random walks in the permutation group )

$\quad$ Let $\mathfrak{S}_n$ denote the permutation group on $n$ symbols. To be clear, let

$$
[n] := \{ \, 1, \dots, n \, \},
$$

and recall that a permutation on $[n]$ is a bijection from $[n]$ to itself. The group structure on the collection of such functions, denoted $\mathfrak{S}_n$, is given by function composition. Below we describe two distinct rules for Markovian dynamics on $\mathfrak{S}_n$. Each set of dynamics can be thought of as arising from forgetting the implicit Cayley graph structure on $\mathfrak{S}_n$: this is the complete graph on $n!$ vertices. Below, we consider adjacency structures on $\mathfrak{S}_n$ distinct from this and from one another. 

$\quad$ Through the adjacency matrix from the previous example, these new graph structures (writing $\sim'$ and $\sim''$ in this example) on $\mathfrak{S}_n$ induce simple random walk dynamics, and so each describes a concrete Markov chain on $\mathfrak{S}_n$, modulo initial conditions.

1. We declare permutations $\sigma$ and $\tau$ adjacent and write $\sigma \sim' \tau$ if $\sigma$ and $\tau$ are related by a transposition of adjacent elements. The $'$-degree of each permutation in this graph is $n-1$.

2. We declare permutations $\sigma$ and $\tau$ adjacent and write $\sigma \sim'' \tau$ if $\sigma$ and $\tau$ are related by a transposition, not necessarily of adjacent elements. The $''$-degree of each permutation in this graph is ${n \choose 2}$. 

These are examples of _distinct_ ergodic random walks on the same state space, namely $\mathfrak{S}_n$, both of which have the uniform measure, let us still denote this $\mu$, as their equilibrium measure. The simulation 'purpose' of both random walks is thus to sample from this uniform measure, and the distinctness of the two walks raises the question of whether one method does the sampling better. 
___

## ... Shannon's model of text ...

_( Menon $\S$ 1.3 )_

$\quad$ The main idea in Shannon's model (of text / written language) is that text is text is a stationary, ergodic stochastic process $(X_t)$ taking values in an alphabet
$$
\mathcal{A} \triangleq \{\, a, \,b,\, c,\, \dots\,,\, z,\, \_ \,\}
$$
This model is easily augmented to include punctuation and case, but we ignore these concepts in the first approximation. Note that we are not assuming that this process is a Markov chain. As discussed, the alphabet is the state space $\mathcal{S}$ from previous sections, but we use this terminology and notation to emphasize the language modeling aspect of the discussion. Note also that stationarity is distinct from the Markov property, in that neither property implies the other.


___

### **Definition.** ( stationary stochastic process )

$\quad$ A discrete time stochastic process $X \equiv (X_t)$ is **stationary** if all abitrarily large, but finite, marginal probabilities are invariant under arbitrary shifts in time. More precisely,

$$
\mathbb{P} (\, X_1 = s_1, \dots, X_n = s_n\,) 
    = 
        \mathbb{P} ( \,X_{1+k} = s_1, \dots, X_{n+k} = s_n \,)
$$

for all positive integers $k$ and $n$.
___

$\quad$ Intuitively, this means that if we were to observe strings of length $n$, their statistics are not changed by shifting them forward in time by an integer $k$. For such processes, the notion of ergodicity again means that _time averages_ are equal to _spatial averages_. From the perspective of observables $\mathcal{O} : \mathcal{S}^n \to \mathbb{R}$, which through the above definition requires have an arbitrarily large 'memory' $n$ of the process, one has
$$
\lim_{T \to \infty} \frac{1}{T} \sum_{t =0 }^{T-1} \mathcal{O}(X_{t+1}, \dots, X_{t+n}) =
\mathbb{E}  \mathcal{O}( X_1, \dots, X_n) 
$$

$\quad$ Menon: _"The above assumptions are introduced to conform to the idea that written text contains many heirarchical structures: letters are joined by phonetic rules to form words, words are linked by the rules of grammar into sentences, sentences are organized into paragraphs, and so on. Strange as it may seem at first sight, these rules can be effectively modeled as a stochastic process. The use of stationary ergodic processes provides us with a definition flexible enough to include heirarchical structures (though these may be complex to write down), and simple enough to computationally test."_

$\quad$ As discussed, written text is modeled by a stationary stochastic process $Y$, whose law is denoted
$$
\mathbb{P}_{\textrm{true}} .
$$
The subscript '$\textrm{true}$' indicates that we view $Y$ not as an approximation to language, but as a stream of 'correctly written' English text, generated in real-time. This idealization is vague, but the main point is to identify the written language $\mathcal{L}$, in this case English, with a concrete mathematical object. In this case, the object is the stationary, stochastic process $Y$. The law $\mathbb{P}_{\textrm{true}}$ of this process is the analogue of $\mu$ in two previous contexts: the uniform measure on $\mathcal{S} = \textsf{square}$, and the uniform measure on $\mathcal{S} = \mathfrak{S}_{52}$. In either previous case, the purpose of the Markov chains involved was to approximately sample from $\mu$. Below, we discuss a family of Markov chains for approximately sampling from $\mu = \mathbb{P}_{\textrm{true}}$. The way in which this sampling happens seems distinct from before, and worth remarking on after introducing the approximating Markov chains. 

$\quad$ In practice, the probabilities relative to $\mathbb{P}_{\textrm{true}}$ below are approximated by mining a large corpus (e.g. the works of Shakespeare) to approximate $\mathbb{P}_{\textrm{true}}(a_1, \dots, a_n)$ for an arbitrary string $(a_1, \dots, a_n) \in \mathcal{A}^n$.

___

### **Example.** ( digram model )

_( or 2-gram model )_

$\quad$ This is a Markov chain with state space $\mathcal{A} = \{ \, a,\, b, \, c, \, \dots \, , \,z, \, \_ \}$. 

$$
Q^{(2)}(x,y) = \mathbb{P}_{\textrm{true}} (\,X_2 = y | X_1 = x\,),
$$

which is to say, we form the character-to-character transition probabilities of a Markov process using the marginals of the full stationary process being approximated. 

___

$\quad$ We can get better approximations by allowing the Markov chain above to retain more of its history. This means defining a Markov process on $\mathcal{A}^n$ for some $ n \geq 2$. When $n=2$, we obtain the so-called trigram model. 



___

### **Example.** ( trigram model ) 

_( or 3-gram model )_

$\quad$ This is a Markov chain with state space $\mathcal{A}^2$, which we style as

$$
\{ aa, \,ab,\,, \dots, az, \dots, \_a, \, \dots, \, \_\,\_ \} 
$$

Let $x = a_1a_2$ and $y = b_1b_2$. In order to form a text of three letters from $x$ and $y$, we must ensure that $x$ and $y$ agree on their overlap, that is when $a_2 = b_1$, so that $x$ and $y$ combine to give us the string $a_1a_2b_2$. With this restriction on $x$ and $y$, we obtain the associated transition matrix

$$
Q^{(3)} (x,y) 
= 
\frac{
\mathbb{P}_{\textrm{true}} ( a_1 a_2 b_2 )
}
{
\mathbb{P}_{\textrm{true}} (a_1 a_2) 
}
$$

and $Q(x,y) =0$ when $a_2 \neq b_1$. 

> Because of the compatability requirement of $a_2 = b_1$, the above Markov chain can still be viewed as a stochastic process taking values in $\mathcal{A}$, it is just no longer a Markov chain from this perspective. This is the sense, however, in which this model is approximating $\mathbb{P}_{\textrm{true}}$.  

___



___

### **Example.** ( $(n+1$)-gram model )

$\quad$ A Markov chain with state space $\mathcal{A}^{n}$. A state $x$, say $x = (a_1, \dots, a_n)$ can be followed by a string $y = (b_1, \dots, b_n)$ only if $a_2 = b_1, a_3 = b_2, \dots, a_n = b_{n-1}$. This induces transition matrix

$$
Q^{(n+1)} (x,y) 
= 
\frac{
\mathbb{P}_{\textrm{true}} ( a_1 a_2 \dots a_n b_n )
}
{
\mathbb{P}_{\textrm{true}} (a_1 a_2 \dots a_n) 
}
$$

as above.

> Following the previous remark, for similar compatibility reasons, this Markov chain can also be interpreted as a stationary stochastic process taking values in $\mathcal{A}$, and as before, this is the sense in which the above process approximates $\mathbb{P}_{\textrm{true}}$.
___

$\quad$ Menon: _"Shannon proved that as $n \to \infty$, the law of text generated by the above Markov approximations converges to the law of the true language. Note however that a good proof may not correspond to a good algorithm. For example, as $n$ increases, the size of the state space ( of the approximating Markov process ) grows exponentially. Thus the dimensions of the transition matrix are $\#\mathcal{A}^{2n}$, and worse yet, the size of the training data ( to determine $Q^{(n+1)}$ by mining ) also expands exponentially."_

$\quad$ _"This follows the rules of everyday language. Once $n$ gets large enough, say $5$ or $6$ in practice, it is much more efficient to make our fundamental unit words, rather than letters, since the number of true words of length $5$ is much lower than the $26^5$ combinations that are possible. This reflects the true nature of the space $\_$ character. In effect, we are still using the digram model, though we have switched to a new 'alphabet' whose fundamental units are words."_

## ... information theory ...

_references_

* Ivan Matic, _Information Theory. Sanov's Theorem_ (notes from July 21, 2018)

$\quad$ Before going further, let us return to the object $\mathbb{P}_{\textrm{true}}$, playing the role of $\mu$ in previous examples. When recalling previous examples, we identifid $\mu$ as the uniform measure on $\mathcal{S}$, in either case. To be formal, $\mu = \mathbb{P}_{\textsf{true}}$ is a probability measure on $\mathcal{S}^\infty \equiv \mathcal{A}^\infty$,
$$
\mathcal{A}^\infty := \{ a \equiv (a_j)_{j=1}^\infty : a_j \in \mathcal{A} \textrm{ for all } j \}\,,
$$
the space of infinite sequences of elements of $\mathcal{A}$. Any discrete-time $\mathcal{A}$-valued stochastic process, given some initial conditions which we ignore, induces a probability measure on $\mathcal{A}^\infty$. This latter space has the interpretation of a 'path-space', as it is meant to contain all possible trajectories of the process. 

$\quad$ The digram model naturally gives rise to a Markov chain with state space $\mathcal{A}$. Let us say that it induces measure $\mathbb{P}_{(2)}$ on $\mathcal{A}^\infty$. The trigram model is defined as a Markov chain with state space $\mathcal{A}^2$, but following the above remark, can be interpreted as a stationary stochastic process taking values in $\mathcal{A}$, inducing a measure $\mathbb{P}_{(3)}$ on the path-space $\mathcal{A}^\infty$. In general, the $(n+1)$-gram model induces measure $\mathbb{P}_{(n+1)}$ on $\mathcal{A}^\infty$. My interpretation of what Menon says at the beginning of the most recent quote, is _"Shannon proved that..."_ $\mathbb{P}_{(n+1)}$ converges to $\mathbb{P}_{\textrm{true}}$, in an appropriate sense. 

> The approximation of $\mathbb{P}_{\textrm{true}}$ by $\mathbb{P}_{(n+1)}$ does _not at all_ seeem to be an approximation of the same nature was is described in Krauth. By this I mean that $\mathcal{A}$-valued Markov chains do not play the same role in each case. In earlier cases, they served as a way to approximately sample from a probability measure on $\mathcal{A}$. In the present context, these objects induce a probability measure on the _path-space_ $\mathcal{A}^\infty$, towards approximating $\mathbb{P}_{\textrm{true}}$, the probability measure on $\mathcal{A}^\infty$ encoding the '$\textrm{true}$' language.  

$\quad$ I want to take a detour through some notes of [Ivan Matic](https://mfe.baruch.cuny.edu/maticivan/), because I think they add to the above discussion, as well as to Menon's upcoming discussion of entropy. Menon: _"the entropy of a random variable describes its uncertainty. In particular, the notion of an optimal code tells us that entropy describes the optimal search procedure to determine the value of $X$ through a series of yes / no questions."_ Matic's notes touch on this optimality, from the perspective of data compression. 

$\quad$ To integrate Matic's perspective, he starts by imagining _"...we want to store some big quantity of data, such as a picture, a text document, or a book. We start with a collection of uncompressed data, and we identify the smallest building blocks of each datum as symbols. The set of all symbols is called the alphabet."_ Matic also denotes this alphabet as $\mathcal{A}$. 

> It feels worth remarking that languages like Chinese are pictoral, and that Chinese characters can be decomposed into 'brushstrokes'. A space character `_` perhaps corresponds to moving to the next character. I think this lends itself to the notion that pictoral data can be decomposed into symbols. For images in general, I think 'contours' become the analogue of brushstrokes. Perhaps an efficient way to use a space character `_` is as denoting the start of a new transparent canvas layer, sitting over the previously drawn layers (copying the functionality of applications like photoshop). My impression of $3$-dim spatial data (e.g. virtual environments for games) is that environment information is efficiently stored in surfaces, the higher-dimensional analogue (by one dimension) of contours. 

> The above decomposition reminds me of something I heard about the clustering algorithm [t-SNE](https://towardsdatascience.com/t-sne-clearly-explained-d84c537f53a): when applid to MNIST, the number of clusters the algorithm found was more than the number of categories (labeled by digits $0 - 9$). Some digits corresponded to multiple clusters _because_ of there being distinct ways to write this digit, with or without a horizontal slash in the case of $7$, for example. In a sense, the clustering worked too well. 

$\quad$ A raw, uncompressed datum will be called a __signal__. The signal is sampled from the measure $\mathbb{P}_{\mathcal{L}}$ on $\mathcal{A}^\infty$ corresponding to abstract language $\mathcal{L}$ with alphabet $\mathcal{A}$. This measure corresponds to a stochastic process $(X_t)_{t \geq 1}$, which we suppose is stationary. Because of the 'optimal encoding' perspective, it is important to be able to quantify the relationship between the lengths of a signal and its encoding. In particular, we assume some upper bound $M$ on the signal length to begin with, modeling signals as elements of $\mathcal{A}^M$ for some $M \in \mathbb{N}$. When referring to $\mathbb{P}_{\mathcal{L}}$ in this context, it is understood as the projection of this measure onto $\mathcal{A}^M$, corresponding to the 'truncated' process $(X_t)_{t = 1}^{M}$. Elements of $\mathcal{A}^M$ will also be called signals (Matic calls them _messages_). 

$\quad$ We begin in an even simpler setting than the above language models. Here, we suppose $\mathbb{P}_{\mathcal{L}}$ corresponds to an i.i.d. sequence of random variables $(X_t)_{t = 1}^M$, where each $X_t$ has the same distribution as some 'model' random variable $X$, corresponding to probability measure $m$ on $\mathcal{A}$. Because $\mathcal{A}$ is finite, so is the space of signals $\mathcal{A}^M$. A lossless data compression is an injective function on this signal space. 

___
### __Definition.__ ( lossless data compression )

A __lossless data compression__ is an injective function
$$
f : \mathcal{A}^M \to \bigcup_{j=1}^N \{ 0 , 1 \} ^j
$$
for some $N \in \mathbb{N}$. 
___

$\quad$ Given some lossless data compression $f$, and for each signal $s \equiv (s_1, \dots, s_M) \in \mathcal{A}^M$, we let $\textsf{length}_f(s)$ denote the length of the signal encoding $f(s)$. Letting $\mathbb{E}$ denote expectation with respect to the 'model' random variable $X$, desirable propereties of the compression $f$ make
$$
\mathbb{E} \, \textsf{length}_f(X) 
$$
as small as possible. Heuristically, this means __(1)__ identifying the signals in $\mathcal{A}^M$ which are _typical_ and _atypical_, according to the sampling $(X_t)_{t=1}^M$, and then __(2)__ using shorter binary sequences to encode typical signals, leaving longer binary sequences for atypical signals. This means the expectation is placing as little probablistic mass as possible on the longest encodings. 

$\quad$ Let us use $m$ to denote the probability measure on $\mathcal{A}$ induced by the random variable $X$, our template for the i.i.d. sequence. Enumerating $\mathcal{A}$ as $\{ a_1, \dots, a_n\}$, let us write $m_i$ for the number $m( \{ a_i \})$. We identify $m$ with the vector $(m_1, \dots, m_n)$, which is the probability mass function of $X$. It is $m$ that determines how 'typical' a given sequence is.

$\quad$ Let $s \equiv (s_1, \dots, s_M) \in \mathcal{A}^M$. Write $\underline{X}$ to denote the random vector $(X_t)_{t=1}^M$. The mutual independence of the $X_t$ implies
$$
\begin{align}
\mathbb{P}( \, \underline{X} = s ) &= \mathbb{P} (X_1 = s_1) \dots \mathbb{P}(X_M = s_M) \\
&= 2^{ \sum_{t=1}^M \log_2\mathbb{P}(X_t = s_t)}\\
&= 2^{ \sum_{i=1}^n n_i \log_2 m_i} \,,
\end{align}
$$
where, in going from the second to third line, we have reindexed the sum over the alphabet, using that $m_i \equiv \mathbb{P}(X = a_i)$, and defining
$$
n_i := \# \{ \, t \in \{ 1, \dots, M \} : X_t = a_i \, \}.
$$
Thus, $n_i$ is the number of times symbol $a_i$ appears in the random signal $\underline{X}$. A typical message is one  in which the frequency of symbol $a_i$ is close to the corresponding probability $m_i$. Thus, for typical messages, we have $n_i \approx m_i M $, and the probability of such a message is approximately
$$
2^{ M \sum_{i=1}^n m_i \log m_i }
$$

___

### __Definition.__ ( entropy )

$\quad$ Let $m$ be a probability measure on finite state space $\mathcal{A}$ with $\# \mathcal{A} = n$, corresponding to probability mass function $(m_1, \dots, m_n)$. The __entropy__ of $m$ is 
$$
\textsf{S}(m) := - \sum_{i=1}^n m_i \log_2 m_i
$$

___

$\quad$ Note that $\textsf{S}(m) \geq 0$, and is $=0$ if and only if $m$ corresponds to a deterministic object, namely a $\delta$-mass at some symbol in the alphabet. In the above setting, we are imagining the length $M$ of the signal is much larger than the size $n$ of the alphabet $\mathcal{A}$. This is so that the law of large numbers has a chance to kick in for each symbol. In this setting, typical messages appear with probability on the order of $2^{-M \textsf{S}(m)}$. When $m$ is non-trivial, the positivity of $\textsf{S}(m)$ implies that typical signals are _exponentially rare_, in the signal length $M$. 

$\quad$ A consquence of the intrinsic high-dimensionality of $\underline{X}$ -- which corresponds to a probability measure on $\mathbb{R}^M$, with $M$ large -- is that signals with symbol frequencies _close_ to the corresponding $m_i$ are very common. The next definition formalizes this notion of closeness. 

___

### __Definition.__ ( $\epsilon$-typical signal )

For fixed $\epsilon > 0$, we say $s \in \mathcal{A}^M$ is __$\epsilon$-typical__ if
$$
\mathbb{P}( \, \underline{X} = s ) \in \left( 2^{-M(\textsf{S}(m) +\epsilon) }, \, 2^{-M(\textsf{S}(m) -\epsilon) } \right) \,,
$$
and the set of all $\epsilon$-typical signals is denoted $\mathcal{A}^M_\epsilon$. 

___

$\quad$ As discussed, the goal is to efficiently compress signals $\underline{X}$ drawn from the measure $m$. The relevance of the above defininition to compression is:

1. we first prove that $\underline{X}$ is $\epsilon$-typical with high probability

2. we will see that the set of typical signals is nonetheless small: the number of all possible signals is $\# \mathcal{A}^M \equiv n^M$, and there are only approximately $2^{M\textsf{S}(m)}$ 

these facts combine to form a compression strategy: when we see a typical message, we begin its binary encoding (towards defining the lossless compression function $f$) with `0`, and we use $M \textsf{S}(m)$ bits to compress it. If the message is atypical, we begin its binary encoding with a `1`, and are more sloppy with how many bits we use to store. The formal statement of the first point is as follows.

___
### __Theorem.__ (Matic, Theorem 1) 

$\quad$ For each $\delta > 0$, there is $M_0 \in \mathbb{N}$ such that for every $M \geq M_0$ there is a lossless data compression $f$ in which the average length of a message satisfies

$$
\mathbb{E} \textsf{length}_f ( \,\underline{X} ) \leq M \textsf{S}(m) + \delta 
$$
___

_( we defer recording the proof )_

Matic: _"The algorithm constructed (in the above proof) creates a lossless compression. It is very effective when the raw data consists of independent components. The `png` format uses a better algorithm."_

$\quad$ To discuss this algorithm, Matic introduces what he calls _types_. These objects were previously introduced as the frequencies $n_i$ above.

___

### __Definition.__ ( the type of a signal )

$\quad$ Given a signal $s \equiv (s_t)_{t=1}^M \in \mathcal{A}^M$, the __type__ of $s$ is the following probability measure on $\mathcal{A}$, constructed through the symbol frequencies of $s$: for $a \in \mathcal{A}$, 
$$
m_s( \{ a \} ) := \frac{1}{M} \sum_{t = 1}^M \mathbf{1}\{ s_t = a \}
$$

___

$\quad$ As a direct consequence of the above definition, two signals $s, \tilde{s} \in \mathcal{A}^M$ have the same type if and only if the coordinates of $\tilde{s}$ form a permutation of the coordinates of $s$. Let $\textsf{types}(\mathcal{A},M)$ denote the collection of all types induced by signals in $\mathcal{A}^M$. The map $\mathcal{A}^M \to \textsf{types}(\mathcal{A},M)$ is effectively a quotient map, and given a probability measure $m \in \textsf{types}( \mathcal{A}, M)$, we let $\textsf{signals}(m)$ denote the collection of signals whose type is $m$:
$$
\textsf{signals}(m) = \{ s \in \mathcal{A}^M : m_s \equiv m \}
$$

___

### __Theorem.__ ( Matic, Theorem 2 )

Consider the set of types, namely $\textsf{types}( \mathcal{A}, M)$, with $\# \mathcal{A} = n$. One has

1. $\#\, \textsf{types}( \mathcal{A},M) = {M + n -1 \choose M }$

2. $\# \, \textsf{types}( \mathcal{A}, M) \leq (M+1)^n$

___

$\quad$ Let $\mu$ be a general probability measure on $\mathcal{A}$, corresponding to a random variable $Y$ and let $\underline{Y} \equiv (Y_t)_{t=1}^M$ be a sequence of i.i.d. random variables. Given fixed signal $s \in \mathcal{A}^M$, one has
$$
\begin{align}
\mathbb{P}( \, \underline{Y} = s ) &= \prod_{t=1}^M \mu( Y_t = s_t) \\
&= 2^{ \sum_{t=1}^M \log_2 \mu( Y_t = s_t ) } \\
&= 2^{ \sum_{a \in \mathcal{A} } \sum_{t =1}^M \mathbf{1} \{ X_t = a \} \log_2 \mu(Y_t = s_t)} \\
&= 2^{ \sum_{a \in \mathcal{A} } \sum_{t =1}^M \mathbf{1} \{ X_t = a \} \log_2 \mu(Y_t = a)} \\ 
&= 2^{ M \left[ \sum_{a \in \mathcal{A} } \left( \frac{1}{M} \sum_{t =1}^M \mathbf{1} \{ X_t = a \} \right) \log_2 \mu(Y_t = a) \right]} \\
&= 2^{ M \left[m_s(a) \log_2 \mu(a) \right] }
\end{align}
$$

The coefficient of $M$ in the exponent directly above can be written in terms of the relative entropy.

_next in section_

* relative entropy or KL divergence

___

_next sections_

* Gibbs measures (Menon $\S$ 1.4)

* derivation of the Gibbs distribution (from principles of information theory, continuing with Menon $\S$ 1.4)

* decoding scrambled text (from a Bayesian perspective, Menon $\S$ 1.5)

* sampling from a Gibbs distribution (Menon $\S$ 1.6)

* the Metropolis algorithm (Menon $\S$ 1.7, also consulting parts of first chapter of Krauth on detailed balance and Metropolis, respectively $\S$ 1.1.4 and $\S$ 1.1.5)

* Markov chains convergence to stationary distribution (Menon $\S$ 1.8, 1.9)

_( For now, omit Krauth $\S$1.1.3 which analyzes the Buffon needle problem. In the long term, it would be nice to practice simulating either the RWs on the symmetric group discussed in a pair of examples, or the $(n+1)$-gram model.  )_

$\quad$ The point of building up to this last section is that it lays the theoretical groundwork for how the MCMC method can be used to sample from a Gibbs distribution, which is an abstraction of the simulations run in the motivating [blog post](http://bit-player.org/2021/three-months-in-monte-carlo). What's next is to describe the Hamiltonian for the Ising model, from which the Gibbs measures and dynamics are automatically defined. 